In [ ]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
os.environ['OPENAI_API_KEY'] = 'your open api key'

In [ ]:
#Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500)

loaders = UnstructuredURLLoaders(urls=["link_one"
"link_two"
                                      ])
data = loaders.load()
len(data)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
#As data is of type documents we can directly use split_documents over split_text in order to
docs = text_splitter.split_documents(data)
len(docs)

In [ ]:
docs[0]

In [ ]:
embeddings = OpenAIEmbeddings()

vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
#storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
query = "What is the Price of Tiago iCNG?"

langchain.debug=True

chain({question":query}, return_only_outputs=True)